# 네이버뉴스 웹 크롤링

In [ ]:
# 필요한 라이브러리 불러오기

import sys             # 시스템 관련
import os              # 시스템 관련
import numpy as np    # 행렬 연산을 위한
import requests       # HTTP 호출
import re             # 정규표현식을 이용한 문자열 매칭
from bs4 import BeautifulSoup   # 데이터 정제
from tqdm.notebook import tqdm  # 반복문 진행과정 시각화
import nltk           # 자연어처리 라이브러리
from konlpy.tag import Okt; okt = Okt() # 한국어 자연어 처리(품사분석)
import matplotlib.pyplot as plt # 그래프
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS  # 워드크라우드

In [ ]:
# 변수 설정

news_list = []        # 제목 + 요약내용 담을 리스트
keyword = '대한항공'     # 검색어
page_num = 1             # 뉴스검색 페이지
ds = '2021.01.01'        # 검색 날짜지정(시작일)
de = '2021.03.31'        # 검색 날짜지정(종료일)

In [ ]:
# 네이버 뉴스 크롤링
# 정해진 날짜내에서 최대 4천건을 검색

last = False           # while 반복문을 돌리기위한 변수설정
pbar = tqdm(total=3990) # tqdm 진행바를 3990에서 끝나도록 설정
while last == False:  # last=True로 될때까지 반복
    url = 'https://search.naver.com/search.naver?&where=news&sm=tab_pge\
    &query={0}&sort=0&photo=0&field=0&pd=3&ds={1}&de={2}&mynews=0\
    &office_type=0&office_section_code=0&nso=so:r,p:,a:all&start={3}'.format(
    keyword,ds,de,str(page_num))           # 뉴스페이지 url, 위에서 설정한 변수들을 .format으로 넣어줌
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}  # 웹크롤링 오류방지를 위해 헤더설정
    
    res = requests.get(url, headers = headers)              # requests를 통해 url의 HTML을 가져옴
    soup = BeautifulSoup(res.text, 'html.parser')           # HTML 페이지 분석
    
    # 뉴스 제목,내용요약 부분이 있는 ul(class:list_news)를 기사별(10개)로 가져온다.
    li_list = soup.find('ul', {'class':'list_news'}).find_all('li', {'id':re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class':'news_area'}) for li in li_list]    # 제목을 가져오기 위한 리스트 반복문
    a_list = [area.find('a', {'class':'news_tit'}) for area in area_list]    
    div_list = [li.find('div', {'class':'dsc_wrap'}) for li in li_list]      # 내용요약을 가져오기 위한 리스트 반복문

    for x,y in zip(a_list, div_list):    # 제목과 내용요약을 news_list에 담는 반복문(기사10개)
        news_list.append(x.get('title'))
        news_list.append(y.text)
        
    page_num += 10   # 페이지 넘김(네이버는 페이지가 1,11,21~증가함)
    pbar.update(10)  # 반복문이 한번 돌때마다 tqdm 진행바를 10 씩 채움
    
    # 마지막 페이지에서 반복문을 끝내기 위해 페이지표기 숫자를 가져옴
    page = soup.find('div', {'class':'sc_page_inner'})
    page_a_list = page.find_all('a')
    pages = [a.text for a in page_a_list]
    
    if page_num == 91:    # 네이버는 페이지가 10이어도 뒷페이지가 자동으로 표시되지 않기때문에 넘김
        continue
    elif page_num == (int(pages[-1])*10-9): # 마지막페이지에서 last=True로 설정
        last = True

pbar.close()          # 반복문이 끝나면 tqdm 진행바를 끝냄
#print(news_list)     # 확인용 출력

# 웹크롤링한 데이터 자연어 처리

In [ ]:
# 크롤링한 기사들을 Okt를 사용하여 품사분석하고 필요없는 품사 제거

results = []    # 최종 결과물을 담을 리스트
for line in news_list:
    intm = okt.pos(line, norm=True, stem=True)    # Okt를 사용해 한국어 형태소 분석(norm:정규화, stem:어간추출)
    
    r = []    # 필요없는 품사를 제외한 형태소들을 담을 리스트
    for word in intm:
        if not word[1] in ['Puctuation', 'Josa', 'Foreign', 'Number', 'Verb']:    # 개인적으로 판단하여 제거하자
            r.append(word[0])                                            # ('단어','품사')형태라 word[0],word[1]
    
    r1 = (' '.join(r)).strip()    # 필요한 형태소들만 다시 합쳐 line화(문장) 만듬
    results.append(r1)

#print(results)    # 확인용 출력  

In [ ]:
# 리스트에 담겨있는 문장들을 하나의 텍스트로 만듬

collect_text = ''

for each_line in results:
    collect_text = collect_text + each_line + '\n'
    
#print(collect_text) # 확인용 출력

In [ ]:
# 하나의 텍스트를 형태소별로 나누고 불용어를 지정하여 걸러내기

tokens_ko = okt.morphs(collect_text)    # 형태소별로 나누기

stop_words = ['.',',','\n','…','(',')','..','개','변','약','...',"'",'"','·','[',']','=','’','-','있다','등','이',
             '들','것','화','원','위해','~']    # 불용어 지정하기

tokens_ko = [each_word for each_word in tokens_ko
            if each_word not in stop_words]         # 불용어 걸러내기

#tokens_ko     # 결과물 확인하면서 불용어 추가할것

# 자연어 처리를 통한 시각화

In [ ]:
# 자연어의 데이터 탐색을 쉽게해주는 nltk.Text
# nltk.Text를 이용해 그래프를 그리거나 워드크라우드를 만들 수 있음

ko = nltk.Text(tokens_ko, name='네이버기사')
ko.vocab().most_common(10)    # 빈도수가 가장 많은 순으로 정리(개수)

In [ ]:
# Matplotlib 그래프에 한글이 문제없이 표기되도록 하는 부분

import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unkown system... sorry~~~~')

In [ ]:
# 빈도수를 y축으로 하는 그래프를 그림

plt.figure(figsize=(15,6))
ko.plot(10)                # (보여줄 단어 개수)
plt.show()

In [ ]:
# 단어의 빈도수를 보여주는 워드크라우드 만들기

data = ko.vocab().most_common(50)    # 빈도수 순으로 최대 50개의 단어를 추려냄

# 딕셔너리화 시킨 후 워드크라우드 만들기
wordcloud = WordCloud(font_path=path,
                     relative_scaling=0.2,
                     background_color='white').generate_from_frequencies(dict(data))

# 워드크라우드를 그리기(Matplotlib)
plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()